> ## Introdução

<img align="left" src="https://img.ibxk.com.br/2020/01/29/29163933423523.jpg"></img>

## COVID-19 esta causando estragos em todo o mundo!
> #### Isto não é novidade pra ninguém, porém, seguem as principais dúvidas:

1. Realmente é a China o país mais afetado até o momento? 
2. Há países da América / Europa em situações mais críticas? 
3. Como esta a situação no Brasil?
4. Como isto se espalhou tanto?
5. Estar de quarentena ajuda?
6. Quais lições aprendemos da China?
7. Precisamos ficar preocupados?

> #### Este artigo é baseados em dados fornecidos por orgãos internacionais que auxiliam a responder as questões acima.

## Referências
> #### Dados
* [Novel Coronavirus (COVID-19) Cases, provided by JHU CSSE](https://github.com/CSSEGISandData/COVID-19)
* [COVID19 Global Forecasting (Week 1)](https://www.kaggle.com/c/covid19-global-forecasting-week-1)
* [Novel Corona Virus 2019 Dataset](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset)
* [COVID-19 Complete Dataset (Updated every 24hrs)](https://www.kaggle.com/imdevskp/corona-virus-report)

> #### Notebooks
* [covid-19-analysis-visualization-comparisons](https://www.kaggle.com/imdevskp/covid-19-analysis-visualization-comparisons)
* [Creating a good Analytics Report](https://www.kaggle.com/jpmiller/creating-a-good-analytics-report)

> #### Dashboards
* [JHU Dashboard Web](https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)
* [JHU Dashbiard Mobile](http://www.arcgis.com/apps/opsdashboard/index.html#/85320e2ea5424dfaaa75ae62e5c06e61)
* [Bing Live Dashboard](https://www.bing.com/covid)

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots

from pathlib import Path

> #### Os dados abaixo são atualizados a cada 24hs.

In [ ]:
cleaned_data = pd.read_csv('../input/corona-virus-report/covid_19_clean_complete.csv', parse_dates=['Date'])
cleaned_data.head()

In [ ]:
#atualizando os headers para facilitar a identificação.
cleaned_data.rename(columns={
                     'Province/State':'uf',
                     'Country/Region':'pais',
                     'Lat':'latitude',
                     'Long': 'longitude',
                     'Date': 'data', 
                     'Confirmed': 'confirmado',
                     'Deaths':'mortes',
                     'Recovered':'curado'
                    }, inplace=True)

# casos 
cases = ['confirmado', 'mortes', 'curado', 'ativos']

# casos = confirmado - mortes - curado
cleaned_data['ativos'] = cleaned_data['confirmado'] - cleaned_data['mortes'] - cleaned_data['curado']

# atualização do nome "Mainland China" para "China"
cleaned_data['pais'] = cleaned_data['pais'].replace('Mainland China', 'China')

# Limpando os dados  
cleaned_data[['uf']] = cleaned_data[['uf']].fillna('')
cleaned_data[cases] = cleaned_data[cases].fillna(0)

data = cleaned_data
cleaned_data.head()

### Total de Registros Coletados

In [ ]:
print(f"Primeiro registro:     {data['data'].min()}")
print(f"Último registro:       {data['data'].max()}")
print(f"Total de dias:         {data['data'].max() - data['data'].min()}")

# Análise de Dados - COVID-19

## [1. CASOS CONFIRMADOS](http://)

**O gráfico abaixo demonstra a evolução dos casos confirmados no Mundo**

In [ ]:
grouped = data.groupby('data')['data', 'confirmado', 'mortes'].sum().reset_index()

fig = px.line(grouped, x="data", y="confirmado", title="Total de infectados no Mundo")
fig.update_layout(
    title="Infectados confirmados no Mundo",
    xaxis_title="Data",
    yaxis_title="Infectados")
fig.show()

1. Parece que o crescimento exponencial da pandemia permanece e isso não é bom.

2. A inclinação da linha no período mais recente é muito alta, tornando as coisas ainda piores.

3. Chegaremos rapidamente ao número de 400 mil infectados em todo o mundo (próximos dias).

**Abaixo, os 20 países com maior número de infectados no Mundo**

In [ ]:
data['uf'] = data['uf'].fillna('')
temp = data[[col for col in data.columns if col != 'uf']]

latest = temp[temp['data'] == max(temp['data'])].reset_index()
latest_grouped = latest.groupby('pais')['confirmado', 'mortes'].sum().reset_index()

fig = px.bar(latest_grouped.sort_values('confirmado', ascending=False)[:20][::1], 
             y='confirmado', x='pais',height=700, orientation='v')
fig.update_layout(
    title="Total de casos confirmados no Mundo",
    xaxis_title="País",
    yaxis_title="Infectados")

fig.show()

1. China (Ásia) e Italia (Europa) são os países mais infectados. 
2. EUA em terceiro lidera na América, ou seja, maior número de casos fora da Europa e Ásia.
3. O Brazil esta na décima sétima posição do rank mundial de casos, liderando o cenário sul-americano.



**Vamos analisar os números destes três países, incluir o Brasil e analisar a evolução mundial.**

In [ ]:
grouped_china = data[data['pais'] == "China"].reset_index()
grouped_china_date = grouped_china.groupby('data')['data', 'confirmado', 'mortes'].sum().reset_index()

grouped_italy = data[data['pais'] == "Italy"].reset_index()
grouped_italy_date = grouped_italy.groupby('data')['data', 'confirmado', 'mortes'].sum().reset_index()

grouped_us = data[data['pais'] == "US"].reset_index()
grouped_us_date = grouped_us.groupby('data')['data', 'confirmado', 'mortes'].sum().reset_index()

grouped_br = data[data['pais'] == "Brazil"].reset_index()
grouped_br_date = grouped_br.groupby('data')['data', 'confirmado', 'mortes'].sum().reset_index()

grouped_rest = data[~data['pais'].isin(['China', 'Italy', 'US', 'Brazil'])].reset_index()
grouped_rest_date = grouped_rest.groupby('data')['data', 'confirmado', 'mortes'].sum().reset_index()

In [ ]:
plot_titles = ['China', 'Italia', 'EUA', 'Brasil', 'Outros Países']

#china
fig = px.line(grouped_china_date, x="data", y="confirmado", 
              title=f"Casos confirmados: {plot_titles[0].upper()}", 
              color_discrete_sequence=['#F61067'],
              height=500
             )
fig.show()

In [ ]:
#italia
fig = px.line(grouped_italy_date, x="data", y="confirmado", 
              title=f"Casos confirmados: {plot_titles[1].upper()}", 
              color_discrete_sequence=['#10eae0'],
              height=500
             )
fig.show()

In [ ]:
#eua
fig = px.line(grouped_us_date, x="data", y="confirmado", 
              title=f"Casos confirmados: {plot_titles[2].upper()}", 
              color_discrete_sequence=['#ea1093'],
              height=500
             )
fig.show()

In [ ]:
#brasil
fig = px.line(grouped_br_date, x="data", y="confirmado", 
              title=f"Casos confirmados: {plot_titles[3].upper()}", 
              color_discrete_sequence=['#e7ea10'],
              height=500
             )
fig.show()

In [ ]:
#mundo
fig = px.line(grouped_rest_date, x="data", y="confirmado", 
              title=f"Casos confirmados: {plot_titles[4].upper()}", 
              color_discrete_sequence=['#10ea67'],
              height=500
             )
fig.show()

1. Nota-se que na China a doença esta controlada deste o final de Fevereiro e o mês de Março. EXCELENTE NOTÍCIA PARA ESTE PAÍS!
2. Diferente da China, a Itália esta sofrendo muito com a pandemia. 
3. O aumento acentuado da Itália é preocupante e os novos dias são realmente cruciais.
4. O pico claro no gráfico dos EUA pode ser o resultado de mais casos sendo testados pela primeira vez.
5. A situação dos EUA também é muito preocupante. Esse aumento nos últimos 2-3 dias é significativo.
6. No Brasil, embora os números sejam menores comparado a outros países, ouve um aumento significativo nos últimos dias.
7. No Brasil, a situação é preocupante, embora com proporções menores que os países citados acima.
8. Nos demais Países, os dados combinados apresentam um crescimento constante no número de infectados.



## Abaixo, uma visão do impacto do vírus na AMÉRICA

In [ ]:
fig = px.choropleth(latest_grouped, locations="pais", 
                    locationmode='country names', color="confirmado", 
                    hover_name="pais", range_color=[1,2000], 
                    color_continuous_scale="portland", 
                    title='Países com casos Confirmados')
#fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
countries = pd.read_csv("../input/countries/america.csv")
countries.head()

america = list(countries['country'])
america_grouped_latest = latest_grouped[latest_grouped['pais'].isin(america)]

america_grouped_latest.head()

In [ ]:
fig = px.choropleth(america_grouped_latest, locations="pais", 
                    locationmode='country names', color="confirmado", 
                    hover_name="pais", range_color=[1,2000], 
                    color_continuous_scale='portland', 
                    title='Mapa de calor dos países infectados da AMÉRICA DO NORTE', scope='north america', height=800)
fig.show()

In [ ]:
fig = px.choropleth(america_grouped_latest, locations="pais", 
                    locationmode='country names', color="confirmado", 
                    hover_name="pais", range_color=[1,2000], 
                    color_continuous_scale='portland', 
                    title='Mapa de calor dos países infectados da AMÉRICA DO SUL', scope='south america', height=800)
fig.show()


** Veja que o COVID-19 esta mais presente na América do Norte, comparado a América do Sul **

1. Na América do Norte o EUA é definitivamente o país com maior número de infectados. 
2. Na América do Sul o Brasil lidera o número de infectados.
3. Aparentemente, iniciativas de bloqueio em fronteiras auxiliam em minimizar a propagação do vírus (devido as diferenças entre países vizinhos).



** Abaixo, vamos analisar o total de casos na América por país **

In [ ]:
fig = px.bar(america_grouped_latest.sort_values('confirmado', ascending=False)[:10][::1], 
             x='pais', y='confirmado', height=700,color_discrete_sequence=['#84DCC6'], orientation='v')
fig.update_layout(
    title="Casos confirmados na América",
    xaxis_title="País",
    yaxis_title="Infectados")
fig.show()

In [ ]:
countries.rename(columns={'country': 'pais', 'population' : 'populacao'}, inplace=True)
countries_data = pd.merge(america_grouped_latest, countries, on='pais', how='inner')
countries_data['p_confirmado'] = countries_data['confirmado'] / countries_data['populacao'] * 100
countries_data['p_mortes'] = countries_data['mortes'] / countries_data['confirmado'] * 100

countries_data.head()

In [ ]:
fig = px.bar(countries_data.sort_values('p_confirmado', ascending=False)[:20][::1], 
             x='pais', y='p_confirmado', height=700,color_discrete_sequence=['#84DCC6'], orientation='v')
fig.update_layout(
    title="Casos confirmados x População por país (América)",
    xaxis_title="País",
    yaxis_title="Infectados em Percentual da População")
fig.show()

1. EUA é o país com maior número de infectados na América.
2. EUA possui o maior índice de infectados em relação ao percentual populacional (0,01% da população)
3. Países pequenos e ilhas do caribe (que estão fora dos notíciários mundiais) também sofrem com a pandemia mundial.
4. Em percentual populacional de infectados e números relativos, o Brasil esta bem atrás de EUA e Canadá, países com grande número populacional, no qual lideram os índices em percentuais. 

## [2. MORTES CONFIRMADAS](http://)

In [ ]:
fig = px.line(grouped, x="data", y="mortes", color_discrete_sequence = ['red'])
fig.update_layout(
    title="Mortes confirmadas no Mundo", 
    xaxis_title="Data",
    yaxis_title="Mortes")
fig.show()

In [ ]:
temp = grouped.melt(id_vars="data", value_vars=['confirmado', 'mortes'], var_name='casos', value_name='count')

fig = px.line(temp, x="data", y="count", color='casos', color_discrete_sequence = ['cyan', 'red'],log_y=True)
fig.update_layout(
    title="Casos confirmados x Mortes confirmadas no MUNDO (Ecala Logarítmica)",
    xaxis_title="Data",
    yaxis_title="Total")

fig.show()

1. Nota-se que há um crescimento rápido do número de mortos nos últimos dias.
2. A proporção do número de **Mortos** acompanha o gráfico de casos **Confirmados**, ou seja, não ocorreram melhoras nos últimos meses.

**Abaixo, os 20 países com maior número de mortes no Mundo**

In [ ]:
data['uf'] = data['uf'].fillna('')
temp = data[[col for col in data.columns if col != 'uf']]

latest = temp[temp['data'] == max(temp['data'])].reset_index()
latest_grouped = latest.groupby('pais')['mortes'].sum().reset_index()

fig = px.bar(latest_grouped.sort_values('mortes', ascending=False)[:20][::1], 
             y='mortes', x='pais',height=700, orientation='v')
fig.update_layout(
    title="Total de Mortes confirmadas no MUNDO POR PAÍS",
    xaxis_title="País",
    yaxis_title="Mortes")

fig.show()

1. Novamente China e Itália lideram os índices, porém, a Itália possui um número maior de óbitos comparado a China (Na Itália, 23% da população é idosa).
2. O Iran, no qual possui números de casos semelhantes ao da Alemanha, também destaca-se em número de mortos. Iniciativas da Alemanha devem ser avaliadas por outros países.

In [ ]:
#china
temp = grouped_china_date.melt(id_vars="data", value_vars=['confirmado', 'mortes'], var_name='casos', value_name='count')

fig = px.line(temp, x="data", y="count", color='casos', color_discrete_sequence = ['cyan', 'red'],log_y=True)
fig.update_layout(
    title="Casos confirmados x Mortes confirmadas na CHINA",
    xaxis_title="Data",
    yaxis_title="Total")

fig.show()

In [ ]:
#italia
temp = grouped_italy_date.melt(id_vars="data", value_vars=['confirmado', 'mortes'], var_name='casos', value_name='count')

fig = px.line(temp, x="data", y="count", color='casos', color_discrete_sequence = ['cyan', 'red'],log_y=True)
fig.update_layout(
    title="Casos confirmados x Mortes confirmadas na ITÁLIA",
    xaxis_title="Data",
    yaxis_title="Total")

fig.show()

In [ ]:
#eua
temp = grouped_us_date.melt(id_vars="data", value_vars=['confirmado', 'mortes'], var_name='casos', value_name='count')

fig = px.line(temp, x="data", y="count", color='casos', color_discrete_sequence = ['cyan', 'red'],log_y=True)
fig.update_layout(
    title="Casos confirmados x Mortes confirmadas na EUA",
    xaxis_title="Data",
    yaxis_title="Total")

fig.show()

In [ ]:
#brazil
temp = grouped_br_date.melt(id_vars="data", value_vars=['confirmado', 'mortes'], var_name='casos', value_name='count')

fig = px.line(temp, x="data", y="count", color='casos', color_discrete_sequence = ['cyan', 'red'],log_y=False)
fig.update_layout(
    title="Casos confirmados x Mortes confirmadas no BRASIL",
    xaxis_title="Data",
    yaxis_title="Total")

fig.show()

In [ ]:
#rest
temp = grouped_rest_date.melt(id_vars="data", value_vars=['confirmado', 'mortes'], var_name='casos', value_name='count')

fig = px.line(temp, x="data", y="count", color='casos', color_discrete_sequence = ['cyan', 'red'],log_y=False)
fig.update_layout(
    title="Casos confirmados x Mortes confirmadas em Outros Países",
    xaxis_title="Data",
    yaxis_title="Total")

fig.show()

1. Nota-se que na China há uma estabilidade no número de mortes oriundas do Covid-19.
2. Itália e EUA vivenciam momentos críticos, com crescimento acentuado no gráfico.
3. No Brasil, o gráfico de mortes ainda mostra um cenário diferente dos dos mais críticos (apesar das mortes relatadas).
4. Em números consolidados, em outros países o crescimento não esta acentuado como EUA e Itália, porém, não estável como a China.

## Abaixo, uma visão do impacto do vírus no número de MORTES NA AMÉRICA

In [ ]:
fig = px.bar(america_grouped_latest.sort_values('mortes', ascending=False)[:10][::1], 
             x='pais', y='mortes', height=700,color_discrete_sequence=['red'], orientation='v')
fig.update_layout(
    title="Mortes confirmadas na América",
    xaxis_title="País",
    yaxis_title="Mortes")
fig.show()

In [ ]:
fig = px.bar(countries_data.sort_values('mortes', ascending=False)[:10][::1], 
             x='pais', y='p_mortes', height=700,color_discrete_sequence=['red'], orientation='v')
fig.update_layout(
    title="Mortes confirmadas x Casos confirmados",
    xaxis_title="País",
    yaxis_title="Mortes em Percentual em relação aos Casos confirmados")
fig.show()

1. Embora o EUA seja o país com maior número de mortes relatadas da América, o país não lidera em valores percentuais.
2. O Brasil é o terceiro país da América com maior número percentual de mortes, trás de Equador e Argentina.
